# Odhad CDPF

## Cíl analýzy

Cílem analýzy je odhad neoklasické agregátní CDPF.

## Data a metoda

Pro odhad byly použity následující hodnoty:

- hrubá přidaná hodnota (Gross Value Added) (převedená na stálé ceny roku 2010 pomocí deflátoru HDP),
- čistý fixní nefinanční kapitál (převedený na stálé ceny roku 2010 pomocí deflátoru kapitálových zásob),
- celkový počet odpracovaných hodin.

In [85]:
import eurostat
import pandas
import statsmodels.formula.api as smf
from statsmodels.tsa.stattools import adfuller
from scipy import stats
import statsmodels.tsa.stattools as ts
import numpy
import seaborn

LOCATIONS = ["CZ", "AT", "EU27_2020"]
BASE_LOCATION = "CZ"
START_YEAR = 1980
END_YEAR = 2020

### Hrubá přidaná hodnota (Gross Value Added)

Hrubá přidaná hodnota je načtena na základě kódu `nama_10_a10`.

In [86]:
data_1_es = eurostat.get_data_df('nama_10_a10')

In [87]:
data_1 = data_1_es
# Value added, gross
data_1 = data_1[data_1["na_item"].isin(["B1G", "D1"])]
# Total, all NACE activities
data_1 = data_1[data_1["nace_r2"] == "TOTAL"]
# Current prices, million euro
data_1 = data_1[data_1["unit"].isin(["CP_MNAC", "PD10_NAC"])]
data_1 = pandas.melt(data_1, id_vars=["unit", "na_item", "geo\\time"], value_vars=list(range(START_YEAR, END_YEAR + 1)),
                     value_name="value", var_name="year")
data_1 = data_1.pivot(index=["geo\\time", "year"], columns=["unit", "na_item"], values="value")
data_1 = data_1.dropna()
data_1[("CP_MNAC", "B1G")] = data_1[("CP_MNAC", "B1G")] * data_1[("PD10_NAC", "B1G")] / 100
data_1[("CP_MNAC", "D1")] = data_1[("CP_MNAC", "D1")] * data_1[("PD10_NAC", "B1G")] / 100
data_1 = data_1.drop([("PD10_NAC", "B1G")], axis=1)
data_1.columns = ['_'.join(col) for col in data_1.columns.values]
data_1 = data_1.rename({"CP_MNAC_B1G": "gross_value_added", "CP_MNAC_D1": "compensation_of_employees"}, axis=1)
data_1 = data_1.reset_index()
data_1.head()

,geo\time,year,gross_value_added,compensation_of_employees
0,AT,1995,130145.540670,73793.930850
1,AT,1996,134886.309636,75360.493182
2,AT,1997,140135.537164,77354.111597
3,AT,1998,146283.001040,80443.033918
4,AT,1999,151754.564940,83621.655376


In [88]:
data_1 = data_1.sort_values(["geo\\time", "year"])
data_1["ln_gross_value_added"] = numpy.log(data_1["gross_value_added"])
data_1["ln_gross_value_added_t_minus_1"] = data_1.groupby("geo\\time")["ln_gross_value_added"].shift(1)
data_1["ln_gross_value_added_diff"] = (data_1["ln_gross_value_added"] - data_1["ln_gross_value_added_t_minus_1"]) / data_1["ln_gross_value_added"]

data_1["ln_compensation_of_employees"] = numpy.log(data_1["compensation_of_employees"])
data_1["ln_compensation_of_employees_t_minus_1"] = data_1.groupby("geo\\time")["ln_compensation_of_employees"].shift(1)
data_1["ln_compensation_of_employees_diff"] = (data_1["ln_compensation_of_employees"] - data_1["ln_compensation_of_employees_t_minus_1"]) / data_1["ln_compensation_of_employees"]

data_1 = data_1[["geo\\time", "year", "ln_gross_value_added", "ln_gross_value_added_diff", "ln_compensation_of_employees", "ln_compensation_of_employees_diff"]]
data_1.tail()


,geo\time,year,ln_gross_value_added,ln_gross_value_added_diff,ln_compensation_of_employees,ln_compensation_of_employees_diff
1073,UK,2015,14.419808,0.002668,13.809209,0.002659
1074,UK,2016,14.478324,0.004042,13.869450,0.004343
1075,UK,2017,14.534647,0.003875,13.929650,0.004322
1076,UK,2018,14.591866,0.003921,13.991750,0.004438
1077,UK,2019,14.648765,0.003884,14.059195,0.004797


In [89]:
data_3_es = eurostat.get_data_df('nama_10_nfa_bs')

In [90]:
data_3 = data_3_es
# Total fixed assets (net)
data_3 = data_3[data_3["asset10"] == "N11N"]
# Total, all activities
data_3 = data_3[data_3["sector"] == "S1"]
# Current fixed prices, millions of national currency
data_3 = data_3[data_3["unit"] == "CP_MNAC"]
data_3 = pandas.melt(data_3, id_vars=["unit", "asset10", "geo\\time"], value_vars=list(range(START_YEAR, END_YEAR + 1)),
                    value_name="total_fixed_assets", var_name="year")
data_3 = data_3.sort_values(["geo\\time", "year"])
data_3 = data_3.dropna()
data_3.head()

,unit,asset10,geo\time,year,total_fixed_assets
420,CP_MNAC,N11N,AT,1995,630857.8
448,CP_MNAC,N11N,AT,1996,659084.4
476,CP_MNAC,N11N,AT,1997,686219.3
504,CP_MNAC,N11N,AT,1998,710697.9
532,CP_MNAC,N11N,AT,1999,737295.8


In [91]:
data_4_es = eurostat.get_data_df('nama_10_nfa_fl')

In [92]:
data_4 = data_4_es
data_4 = data_4[data_4["asset10"] == "N11G"]
data_4 = data_4[data_4["nace_r2"] == "TOTAL"]
data_4 = data_4[data_4["unit"] == "PD10_NAC"]
data_4 = pandas.melt(data_4, id_vars=["unit", "asset10", "geo\\time"], value_vars=list(range(START_YEAR, END_YEAR + 1)),
                    value_name="capital_stock_deflator", var_name="year")
data_4 = data_4.sort_values(["geo\\time", "year"])
data_4 = data_4.dropna()

In [93]:
data_3 = data_3.merge(data_4, on=["geo\\time", "year"])
data_3["total_fixed_assets"] = data_3["total_fixed_assets"] * data_3["capital_stock_deflator"]
data_3["ln_total_fixed_assets"] = numpy.log(data_3["total_fixed_assets"])
data_3["ln_total_fixed_assets_t_minus_1"] = data_3.groupby("geo\\time")["ln_total_fixed_assets"].shift(1)
data_3["ln_total_fixed_assets_diff"] = (data_3["ln_total_fixed_assets"] - data_3["ln_total_fixed_assets_t_minus_1"]) / data_3["ln_total_fixed_assets"]
data_3 = data_3[["geo\\time", "year", "ln_total_fixed_assets", "ln_total_fixed_assets_diff"]]
data_3.head()

,geo\time,year,ln_total_fixed_assets,ln_total_fixed_assets_diff
0,AT,1995,17.736362,NaN
1,AT,1996,17.789630,0.002994
2,AT,1997,17.841370,0.002900
3,AT,1998,17.884289,0.002400
4,AT,1999,17.926085,0.002332


In [94]:
data = pandas.merge(data_1, data_3, on=["geo\\time", "year"])
data = data.dropna()
data = data[["ln_gross_value_added_diff", "ln_compensation_of_employees_diff", "ln_total_fixed_assets_diff", "geo\\time", "year"]]
data = data[data["geo\\time"] == "AT"]
data.head()

,ln_gross_value_added_diff,ln_compensation_of_employees_diff,ln_total_fixed_assets_diff,geo\time,year
1,0.003029,0.001871,0.002994,AT,1996
2,0.003222,0.002320,0.002900,AT,1997
3,0.003610,0.003467,0.002400,AT,1998
4,0.003078,0.003419,0.002332,AT,1999
5,0.005311,0.004172,0.003214,AT,2000


In [95]:
data["const"] = 1

import statsmodels.api as sm
mod = sm.OLS(data["ln_gross_value_added_diff"], data[["const", "ln_compensation_of_employees_diff", "ln_total_fixed_assets_diff"]])
res = mod.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                OLS Regression Results                               
=====================================================================================
Dep. Variable:     ln_gross_value_added_diff   R-squared:                       0.506
Model:                                   OLS   Adj. R-squared:                  0.459
Method:                        Least Squares   F-statistic:                     10.76
Date:                       Tue, 14 Dec 2021   Prob (F-statistic):           0.000608
Time:                               21:57:36   Log-Likelihood:                 133.44
No. Observations:                         24   AIC:                            -260.9
Df Residuals:                             21   BIC:                            -257.3
Df Model:                                  2                                         
Covariance Type:                   nonrobust                                         
=====================================================================================================
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
const                                 0.0006      0.001      0.664      0.514      -0.001       0.003
ln_compensation_of_employees_diff     0.7704      0.227      3.395      0.003       0.299       1.242
ln_total_fixed_assets_diff            0.0877      0.385      0.228      0.822      -0.713       0.888
==============================================================================
Omnibus:                       11.353   Durbin-Watson:                   1.293
Prob(Omnibus):                  0.003   Jarque-Bera (JB):                9.665
Skew:                          -1.268   Prob(JB):                      0.00797
Kurtosis:                       4.798   Cond. No.                     2.05e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.05e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [96]:
data["resid"] = res.resid
data["TFP3"] = data["resid"] + res.params[0]
data["%"] =  data["TFP3"] / data["ln_gross_value_added_diff"]
# data = data[data["year"] != 2009]
data["%"].mean()


0.994143380197645